In [1]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from pathlib import Path
from firebase_admin import firestore
import datetime
import numpy as np

In [2]:
# Fetch the service account key JSON file contents
filename = './key/digital-education-b7441-firebase-adminsdk-o3tx7-26bb9a352d.json'
cred = credentials.Certificate(filename)
default_app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [3]:
def load_df(name):
    col = list(db.collection(name).stream())
    col_dict = list(map(lambda x: x.to_dict(), col))
    df = pd.DataFrame(col_dict)
    return df

In [4]:
df_users = load_df('users')
df_users = df_users.rename(columns={'user':'uuid', 'timestamp':'time'})
df_users.head()

,language,age,uuid,username,time
0,DE,23,1f46a7c6-dc11-4f4c-8c22-6f2081e42629,Arne,NaN
1,EN,23,e70c0a67-906c-4e67-95ba-000640f02168,FDNJK,NaN
2,EN,21,257199b2-db6c-43aa-8ec3-57cadf2aaf2b,Bonjez,1.669555e+12
3,EN,23,f76b3145-4588-497b-bb2c-ca5a4e27e300,asdf,1.669558e+12
4,EN,12,f53a4df6-8e30-48da-814c-bd44dc77ddbb,ffds,1.669556e+12


In [5]:
df_logs = load_df('logs')
df_logs = df_logs.rename(columns={'user':'uuid', 'timestamp':'time'})

df_logs.head()

,currentactivity,cause,timespent,part,uuid,time,nextactivity,group
0,3.0,screenChange,0.126,Problem Solving,db55aeb4-c195-4626-afd9-69d340affd89,1669549050242,4,NaN
1,39.0,screenChange,2.129,Instruction,3ee60990-2dfd-4e6b-8890-02b673b9495e,1669558356144,40,NaN
2,24.0,screenChange,0.240,Instruction,7c4e5ae8-9c49-4c49-88db-06bc175970e9,1669543840617,25,NaN
3,38.0,screenChange,0.263,Instruction,8c0ddd14-542b-4964-8507-85462764d2b8,1669552163383,39,NaN
4,44.0,screenChange,0.078,Instruction,4a21dd68-af05-4de6-9088-f98df0b7b862,1669558958868,45,NaN


In [6]:
df_boards = load_df('boards')
df_boards.head()

,height,uuid,time,id,width,part,slide,gridString
0,5,cbeb3138-f198-4e35-8f5f-39008795c2b1,1669560021029,0,6,ps,4,111111111111111111111111111111
1,1,d15009a1-d04d-460c-a21d-6e825acfcf08,1669560422441,0,4,ps,15,0010
2,1,d343ee00-e623-49fb-89b6-a5d0f7e8aa5d,1669559694586,2,9,ps,10,000000010
3,1,6c890410-2b93-4e90-866c-2a38db21ce00,1669570566223,2,4,ps,14,0010
4,1,c8a2dd44-54e8-4f35-be96-0ae132b568d9,1669560620462,3,4,ps,14,0010


In [7]:
df_feedback = load_df('feedbacks')
df_feedback.head()

,part,message,uuid,time,slide
0,ps,Ez,4a893c00-09e0-4df3-8256-745a2f5056a4,1669558720213,5
1,ps,Haha,54619c2f-fda5-4391-a12b-423070a17d15,1669561253981,12
2,ps,gcd,a5590fdc-7172-4b79-ba5a-4d4b06647330,1669558477287,16
3,ps,Ich habe eine 6 gezeichnet hihihi,c8a2dd44-54e8-4f35-be96-0ae132b568d9,1669560552691,5
4,ps,Bonds,1f883dd0-fecd-46b9-83e0-30ae7d1427a3,1669558857833,5


In [8]:
df_logs.groupby('uuid').head()

,currentactivity,cause,timespent,part,uuid,time,nextactivity,group
0,3.0,screenChange,0.126,Problem Solving,db55aeb4-c195-4626-afd9-69d340affd89,1669549050242,4,NaN
1,39.0,screenChange,2.129,Instruction,3ee60990-2dfd-4e6b-8890-02b673b9495e,1669558356144,40,NaN
2,24.0,screenChange,0.240,Instruction,7c4e5ae8-9c49-4c49-88db-06bc175970e9,1669543840617,25,NaN
3,38.0,screenChange,0.263,Instruction,8c0ddd14-542b-4964-8507-85462764d2b8,1669552163383,39,NaN
4,44.0,screenChange,0.078,Instruction,4a21dd68-af05-4de6-9088-f98df0b7b862,1669558958868,45,NaN
...,...,...,...,...,...,...,...,...
2239,1.0,screenChange,0.466,Problem Solving,f23aa3c4-f367-4ba1-841d-40abb619a5bb,1669564318024,2,NaN
2247,2.0,screenChange,0.093,Problem Solving,dd5506dc-abda-4075-b12b-95ae00324d36,1669549538794,3,NaN
2249,2.0,screenChange,0.165,Problem Solving,a27da6ba-0e1c-4457-a82a-bba376ed6a93,1669550319354,3,NaN
2259,NaN,screenChange,0.263,NaN,a3f3d41a-d5d5-497e-9ea0-43c12dbbc3ec,1669543522017,2,Problem Solving


In [9]:
def convert_timestamp(x):
    if not np.isnan(x):
        return datetime.datetime.fromtimestamp(x/1000)
    else:
        return x

In [28]:
def query_user(uuid):
    user_data = df_users[df_users.uuid == uuid]

    username = user_data.username.iloc[0]
    registered_at = convert_timestamp(user_data.time.iloc[0])
    language = user_data.language.iloc[0]
    age = user_data.age.iloc[0]

    print(f'querying data for user {username}, uuid: {uuid}, age {age}, language: {language}, registered at {registered_at}')
    user_logs = df_logs[df_logs.uuid == uuid]
    user_feedbacks = df_feedback[df_feedback.uuid == uuid]
    user_boards = df_boards[df_boards.uuid == uuid]

    user_history = pd.concat([user_logs, user_feedbacks, user_boards],sort=False).sort_values('time')
    user_history['time'] = user_history['time'].apply(lambda x: convert_timestamp(x))
    user_history = user_history.set_index('time')

    return user_data, user_history

In [29]:
test_uuid = df_users[df_users.username == 'Der Betamann']['uuid'].iloc[0]

data, hist = query_user(test_uuid)
hist.head(64)
hist.to_csv('test.csv')

querying data for user Der Betamann, uuid: 6c890410-2b93-4e90-866c-2a38db21ce00, age 69, language: EN, registered at 2022-11-27 18:35:24.366000
